# Data cleaning and pre-processing

This notebook shows the complete process followed to transform the original data set in a new one that contains the business variables with to be analyized in the next phase of the project.

* Data was readed from the provided file `datos.csv` a 3.6 gigabytes CSV file with 36 variables about 10.651.775 bank operations.
* All the variables were converted to its correct type: date, character, categorical or numeric.
* New variables were derived from the original ones
* Finally the data was normalized and centered around it's mean, so the K-means algorithm could calculate the distance between each observation correctly.

The resulting data set was stored in the Apache Hadoop Distributed File System running on the cloud ready to be analyzed with Spark.

In [42]:
# data.table R package provides all the needed data wrangling features to deal with big data files
library('data.table')

In [43]:
# Reading the data file
DT <- fread('./sample.csv', encoding='Latin-1', na.strings=c("","NA"), sep=",")

In [44]:
# First two rows of the new data table
head(DT, 2)

,ANO,MES,DIA,OP_ADQUIRENTE,ADQUIERENTE,DES_TIPO_ADQUIRENTE,OP_EMISOR,EMISOR,DES_TIPO_EMISOR,DES_AMBITO,OP_IDENT_TERMINAL,OP_COD_POST_COMERCIO,DES_PROVINCIA,LOCALIDAD,OP_COD_PAIS_COMERCIO,DES_MARCA,DES_GAMA,DES_PRODUCTO,TIPO_TARJETA,DES_CREDEB,DES_CLASE_OPERACION,DES_PAGO,DES_RESULTADO,PER_ID_PERSONA,PER_TIPO_PERS,PER_FECHA_ALTA,OF_COD_POST,PER_COD_PAIS_NAC,OF_COD_PAIS_RES,PER_ID_SEXO,PER_EST_CIVIL,PER_MARCA_EMP,PER_MARCA_FALL,PER_FECHA_NAC,NOPER,IMPOPER
1,2016,03,26,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,EUNDITVNRWGASN0VCFMK,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,HWG2BTMLK1WMRVJT3TMU,F,19840926,29002,ESP,ESP,F,C,0,0,19410304,1,30
2,2016,03,03,BM3MV1QJ1RWI6XB8W36S,Q4SRXYQNPFB8ST2BCSLT,EURO 6000,X4W6L75KAK6TKQFMYXGJ,T08GEJ1FFLLW5WK82Z0M,EURO 6000,On us,HDOHII3H5SMPCZIJI07V,99999,NO EXISTE LA PROVINCIA,NA,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,P,DÃ©bito,Reintegros,Debito,OK,S72O8TEU3YWJFJLXJO7Y,F,19931218,47006,ESP,ESP,F,C,0,0,19451121,1,200


In [45]:
# All the variables were readed as characters by fread data table function so we need to recode them.
as.data.frame(sapply(DT,class))

,"sapply(DT, class)"
ANO,character
MES,character
DIA,character
OP_ADQUIRENTE,character
ADQUIERENTE,character
DES_TIPO_ADQUIRENTE,character
OP_EMISOR,character
EMISOR,character
DES_TIPO_EMISOR,character
DES_AMBITO,character


In [46]:
# Define a new variable FECHA with the operation date
DT[,FECHA:=as.Date(paste(ANO, MES, DIA, sep="-" ), tz = "Europe/Madrid")]

# Convert to date format both PER_FECHA_NAC and PER_FECHA_ALTA
DT[,PER_FECHA_NAC:=as.Date(PER_FECHA_NAC, format = "%Y%m%d", tz = "Europe/Madrid")]
DT[,PER_FECHA_ALTA:=as.Date(PER_FECHA_ALTA, format = "%Y%m%d", tz = "Europe/Madrid")]

,ANO,MES,DIA,OP_ADQUIRENTE,ADQUIERENTE,DES_TIPO_ADQUIRENTE,OP_EMISOR,EMISOR,DES_TIPO_EMISOR,DES_AMBITO,OP_IDENT_TERMINAL,OP_COD_POST_COMERCIO,DES_PROVINCIA,LOCALIDAD,OP_COD_PAIS_COMERCIO,DES_MARCA,DES_GAMA,DES_PRODUCTO,TIPO_TARJETA,DES_CREDEB,DES_CLASE_OPERACION,DES_PAGO,DES_RESULTADO,PER_ID_PERSONA,PER_TIPO_PERS,PER_FECHA_ALTA,OF_COD_POST,PER_COD_PAIS_NAC,OF_COD_PAIS_RES,PER_ID_SEXO,PER_EST_CIVIL,PER_MARCA_EMP,PER_MARCA_FALL,PER_FECHA_NAC,NOPER,IMPOPER,FECHA
1,2016,03,26,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,EUNDITVNRWGASN0VCFMK,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,HWG2BTMLK1WMRVJT3TMU,F,19840926,29002,ESP,ESP,F,C,0,0,19410304,1,30,2016-03-26
2,2016,03,03,BM3MV1QJ1RWI6XB8W36S,Q4SRXYQNPFB8ST2BCSLT,EURO 6000,X4W6L75KAK6TKQFMYXGJ,T08GEJ1FFLLW5WK82Z0M,EURO 6000,On us,HDOHII3H5SMPCZIJI07V,99999,NO EXISTE LA PROVINCIA,NA,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,P,DÃ©bito,Reintegros,Debito,OK,S72O8TEU3YWJFJLXJO7Y,F,19931218,47006,ESP,ESP,F,C,0,0,19451121,1,200,2016-03-03
3,2016,03,20,X4HO0YVLHY4IY6I77135,KTXGB1YGLNA5A3MBQWPC,ServiRed,S33T9PXGBMI7GUX051OC,JFD7RA18JJ6YI9L720RE,EURO 6000,Inter-Sistemas,Q7MULZRDPH24847MYMLF,46680,Valencia,VALENCIA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,AJ6YNSIZOH62VR52XFDB,F,NA,0,NA,NA,M,X,0,0,0,1,50,2016-03-20
4,2016,01,05,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,9GETG4KKGZXQ8PKZSIXT,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,ONF35G4DR4APTKVHMG5A,F,19820720,11100,ESP,ESP,F,C,0,0,19660322,1,20,2016-01-05
5,2016,03,12,8KZOFPY0TDQH5XW7LV8P,SZBCQF5YAO4W2N3ROXI4,EURO 6000,RI7WGGJYVBDG339OG6O3,Q3POXRTGOT0XT7GJTXI8,ServiRed,Inter-Sistemas,SO7T7EIFPQH2FUYZULYC,2005,Albacete,ALBACETE,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2,150,2016-03-12
6,2016,01,05,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,LWM3BYLE6N04FKCD5TYG,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,CrÃ©dito,Reintegros,Debito,OK,8TYVA5CA0VNQ41TFC7QQ,F,19860526,29620,NA,ESP,M,S,0,0,19771204,1,20,2016-01-05
7,2016,01,20,T0HUIQJLCS3MSSJB229H,6OWQM98TUB4TLU818SPA,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,Intra-Sistema,VX95PMMDEA8OYIBHOXCA,18003,Granada,GRANADA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,2XLFJUUN9VKDAVMRBU4Z,F,19940121,29500,ESP,ESP,F,S,0,0,19940121,1,30,2016-01-20
8,2016,01,03,DNYR3O1F7XM5GOOOSLLB,IDFXVIA8S1D7PND1QNPF,ServiRed,S33T9PXGBMI7GUX051OC,JFD7RA18JJ6YI9L720RE,EURO 6000,Inter-Sistemas,Q7MULZRDPH24847MYMLF,31010,Navarra,NAVARRA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,F0NZ5BBOJGSRAVY0LYUC,F,NA,0,NA,NA,M,X,0,0,0,1,20,2016-01-03
9,2016,02,05,33LCZG4TQMVGQQ2IOQFE,BMENWJCG94XKFNY6CJ55,ServiRed,NKIE64TALWTZ1UHGS17T,X7ZJC37USW0KQYKO68XB,EURO 6000,Inter-Sistemas,BUSM8IDXPSIC74P5S6QU,8930,Barcelona,BARCELONA,ESP,Visa,EstÃ¡ndar,Visa,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,50,2016-02-05
10,2016,01,29,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,5UO3QCLR3WPCSYO3K8JB,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,T30ED28AF5MMY9PJU0JM,F,19770209,29001,ESP,ESP,F,S,0,0,19530328,1,60,2016-01-29


,ANO,MES,DIA,OP_ADQUIRENTE,ADQUIERENTE,DES_TIPO_ADQUIRENTE,OP_EMISOR,EMISOR,DES_TIPO_EMISOR,DES_AMBITO,OP_IDENT_TERMINAL,OP_COD_POST_COMERCIO,DES_PROVINCIA,LOCALIDAD,OP_COD_PAIS_COMERCIO,DES_MARCA,DES_GAMA,DES_PRODUCTO,TIPO_TARJETA,DES_CREDEB,DES_CLASE_OPERACION,DES_PAGO,DES_RESULTADO,PER_ID_PERSONA,PER_TIPO_PERS,PER_FECHA_ALTA,OF_COD_POST,PER_COD_PAIS_NAC,OF_COD_PAIS_RES,PER_ID_SEXO,PER_EST_CIVIL,PER_MARCA_EMP,PER_MARCA_FALL,PER_FECHA_NAC,NOPER,IMPOPER,FECHA
1,2016,03,26,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,EUNDITVNRWGASN0VCFMK,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,HWG2BTMLK1WMRVJT3TMU,F,19840926,29002,ESP,ESP,F,C,0,0,1941-03-04,1,30,2016-03-26
2,2016,03,03,BM3MV1QJ1RWI6XB8W36S,Q4SRXYQNPFB8ST2BCSLT,EURO 6000,X4W6L75KAK6TKQFMYXGJ,T08GEJ1FFLLW5WK82Z0M,EURO 6000,On us,HDOHII3H5SMPCZIJI07V,99999,NO EXISTE LA PROVINCIA,NA,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,P,DÃ©bito,Reintegros,Debito,OK,S72O8TEU3YWJFJLXJO7Y,F,19931218,47006,ESP,ESP,F,C,0,0,1945-11-21,1,200,2016-03-03
3,2016,03,20,X4HO0YVLHY4IY6I77135,KTXGB1YGLNA5A3MBQWPC,ServiRed,S33T9PXGBMI7GUX051OC,JFD7RA18JJ6YI9L720RE,EURO 6000,Inter-Sistemas,Q7MULZRDPH24847MYMLF,46680,Valencia,VALENCIA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,AJ6YNSIZOH62VR52XFDB,F,NA,0,NA,NA,M,X,0,0,NA,1,50,2016-03-20
4,2016,01,05,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,9GETG4KKGZXQ8PKZSIXT,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,ONF35G4DR4APTKVHMG5A,F,19820720,11100,ESP,ESP,F,C,0,0,1966-03-22,1,20,2016-01-05
5,2016,03,12,8KZOFPY0TDQH5XW7LV8P,SZBCQF5YAO4W2N3ROXI4,EURO 6000,RI7WGGJYVBDG339OG6O3,Q3POXRTGOT0XT7GJTXI8,ServiRed,Inter-Sistemas,SO7T7EIFPQH2FUYZULYC,2005,Albacete,ALBACETE,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2,150,2016-03-12
6,2016,01,05,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,LWM3BYLE6N04FKCD5TYG,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,CrÃ©dito,Reintegros,Debito,OK,8TYVA5CA0VNQ41TFC7QQ,F,19860526,29620,NA,ESP,M,S,0,0,1977-12-04,1,20,2016-01-05
7,2016,01,20,T0HUIQJLCS3MSSJB229H,6OWQM98TUB4TLU818SPA,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,Intra-Sistema,VX95PMMDEA8OYIBHOXCA,18003,Granada,GRANADA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,2XLFJUUN9VKDAVMRBU4Z,F,19940121,29500,ESP,ESP,F,S,0,0,1994-01-21,1,30,2016-01-20
8,2016,01,03,DNYR3O1F7XM5GOOOSLLB,IDFXVIA8S1D7PND1QNPF,ServiRed,S33T9PXGBMI7GUX051OC,JFD7RA18JJ6YI9L720RE,EURO 6000,Inter-Sistemas,Q7MULZRDPH24847MYMLF,31010,Navarra,NAVARRA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,F0NZ5BBOJGSRAVY0LYUC,F,NA,0,NA,NA,M,X,0,0,NA,1,20,2016-01-03
9,2016,02,05,33LCZG4TQMVGQQ2IOQFE,BMENWJCG94XKFNY6CJ55,ServiRed,NKIE64TALWTZ1UHGS17T,X7ZJC37USW0KQYKO68XB,EURO 6000,Inter-Sistemas,BUSM8IDXPSIC74P5S6QU,8930,Barcelona,BARCELONA,ESP,Visa,EstÃ¡ndar,Visa,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,50,2016-02-05
10,2016,01,29,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,5UO3QCLR3WPCSYO3K8JB,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,T30ED28AF5MMY9PJU0JM,F,19770209,29001,ESP,ESP,F,S,0,0,1953-03-28,1,60,2016-01-29


,ANO,MES,DIA,OP_ADQUIRENTE,ADQUIERENTE,DES_TIPO_ADQUIRENTE,OP_EMISOR,EMISOR,DES_TIPO_EMISOR,DES_AMBITO,OP_IDENT_TERMINAL,OP_COD_POST_COMERCIO,DES_PROVINCIA,LOCALIDAD,OP_COD_PAIS_COMERCIO,DES_MARCA,DES_GAMA,DES_PRODUCTO,TIPO_TARJETA,DES_CREDEB,DES_CLASE_OPERACION,DES_PAGO,DES_RESULTADO,PER_ID_PERSONA,PER_TIPO_PERS,PER_FECHA_ALTA,OF_COD_POST,PER_COD_PAIS_NAC,OF_COD_PAIS_RES,PER_ID_SEXO,PER_EST_CIVIL,PER_MARCA_EMP,PER_MARCA_FALL,PER_FECHA_NAC,NOPER,IMPOPER,FECHA
1,2016,03,26,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,EUNDITVNRWGASN0VCFMK,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,HWG2BTMLK1WMRVJT3TMU,F,1984-09-26,29002,ESP,ESP,F,C,0,0,1941-03-04,1,30,2016-03-26
2,2016,03,03,BM3MV1QJ1RWI6XB8W36S,Q4SRXYQNPFB8ST2BCSLT,EURO 6000,X4W6L75KAK6TKQFMYXGJ,T08GEJ1FFLLW5WK82Z0M,EURO 6000,On us,HDOHII3H5SMPCZIJI07V,99999,NO EXISTE LA PROVINCIA,NA,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,P,DÃ©bito,Reintegros,Debito,OK,S72O8TEU3YWJFJLXJO7Y,F,1993-12-18,47006,ESP,ESP,F,C,0,0,1945-11-21,1,200,2016-03-03
3,2016,03,20,X4HO0YVLHY4IY6I77135,KTXGB1YGLNA5A3MBQWPC,ServiRed,S33T9PXGBMI7GUX051OC,JFD7RA18JJ6YI9L720RE,EURO 6000,Inter-Sistemas,Q7MULZRDPH24847MYMLF,46680,Valencia,VALENCIA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,AJ6YNSIZOH62VR52XFDB,F,NA,0,NA,NA,M,X,0,0,NA,1,50,2016-03-20
4,2016,01,05,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,9GETG4KKGZXQ8PKZSIXT,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,ONF35G4DR4APTKVHMG5A,F,1982-07-20,11100,ESP,ESP,F,C,0,0,1966-03-22,1,20,2016-01-05
5,2016,03,12,8KZOFPY0TDQH5XW7LV8P,SZBCQF5YAO4W2N3ROXI4,EURO 6000,RI7WGGJYVBDG339OG6O3,Q3POXRTGOT0XT7GJTXI8,ServiRed,Inter-Sistemas,SO7T7EIFPQH2FUYZULYC,2005,Albacete,ALBACETE,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2,150,2016-03-12
6,2016,01,05,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,LWM3BYLE6N04FKCD5TYG,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,CrÃ©dito,Reintegros,Debito,OK,8TYVA5CA0VNQ41TFC7QQ,F,1986-05-26,29620,NA,ESP,M,S,0,0,1977-12-04,1,20,2016-01-05
7,2016,01,20,T0HUIQJLCS3MSSJB229H,6OWQM98TUB4TLU818SPA,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,Intra-Sistema,VX95PMMDEA8OYIBHOXCA,18003,Granada,GRANADA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,2XLFJUUN9VKDAVMRBU4Z,F,1994-01-21,29500,ESP,ESP,F,S,0,0,1994-01-21,1,30,2016-01-20
8,2016,01,03,DNYR3O1F7XM5GOOOSLLB,IDFXVIA8S1D7PND1QNPF,ServiRed,S33T9PXGBMI7GUX051OC,JFD7RA18JJ6YI9L720RE,EURO 6000,Inter-Sistemas,Q7MULZRDPH24847MYMLF,31010,Navarra,NAVARRA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,F0NZ5BBOJGSRAVY0LYUC,F,NA,0,NA,NA,M,X,0,0,NA,1,20,2016-01-03
9,2016,02,05,33LCZG4TQMVGQQ2IOQFE,BMENWJCG94XKFNY6CJ55,ServiRed,NKIE64TALWTZ1UHGS17T,X7ZJC37USW0KQYKO68XB,EURO 6000,Inter-Sistemas,BUSM8IDXPSIC74P5S6QU,8930,Barcelona,BARCELONA,ESP,Visa,EstÃ¡ndar,Visa,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,50,2016-02-05
10,2016,01,29,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,5UO3QCLR3WPCSYO3K8JB,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,T30ED28AF5MMY9PJU0JM,F,1977-02-09,29001,ESP,ESP,F,S,0,0,1953-03-28,1,60,2016-01-29


In [47]:
# Convert character variables into categorical (factor) variables in order to speed data processing
variables <- c('ANO','MES','DIA','OP_ADQUIRENTE','DES_TIPO_EMISOR','DES_PROVINCIA', 'DES_TIPO_ADQUIRENTE', 'DES_AMBITO', 'OP_COD_PAIS_COMERCIO','DES_MARCA','DES_GAMA','DES_PRODUCTO', 'TIPO_TARJETA', 'DES_CREDEB','DES_CLASE_OPERACION', 'DES_PAGO','DES_RESULTADO','PER_TIPO_PERS','PER_COD_PAIS_NAC', 'OF_COD_PAIS_RES','PER_ID_SEXO','PER_EST_CIVIL','PER_MARCA_EMP','PER_MARCA_FALL')
DT[,(variables):=lapply(.SD, as.factor),.SDcols=variables]
rm(variables)

,ANO,MES,DIA,OP_ADQUIRENTE,ADQUIERENTE,DES_TIPO_ADQUIRENTE,OP_EMISOR,EMISOR,DES_TIPO_EMISOR,DES_AMBITO,OP_IDENT_TERMINAL,OP_COD_POST_COMERCIO,DES_PROVINCIA,LOCALIDAD,OP_COD_PAIS_COMERCIO,DES_MARCA,DES_GAMA,DES_PRODUCTO,TIPO_TARJETA,DES_CREDEB,DES_CLASE_OPERACION,DES_PAGO,DES_RESULTADO,PER_ID_PERSONA,PER_TIPO_PERS,PER_FECHA_ALTA,OF_COD_POST,PER_COD_PAIS_NAC,OF_COD_PAIS_RES,PER_ID_SEXO,PER_EST_CIVIL,PER_MARCA_EMP,PER_MARCA_FALL,PER_FECHA_NAC,NOPER,IMPOPER,FECHA
1,2016,03,26,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,EUNDITVNRWGASN0VCFMK,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,HWG2BTMLK1WMRVJT3TMU,F,1984-09-26,29002,ESP,ESP,F,C,0,0,1941-03-04,1,30,2016-03-26
2,2016,03,03,BM3MV1QJ1RWI6XB8W36S,Q4SRXYQNPFB8ST2BCSLT,EURO 6000,X4W6L75KAK6TKQFMYXGJ,T08GEJ1FFLLW5WK82Z0M,EURO 6000,On us,HDOHII3H5SMPCZIJI07V,99999,NO EXISTE LA PROVINCIA,NA,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,P,DÃ©bito,Reintegros,Debito,OK,S72O8TEU3YWJFJLXJO7Y,F,1993-12-18,47006,ESP,ESP,F,C,0,0,1945-11-21,1,200,2016-03-03
3,2016,03,20,X4HO0YVLHY4IY6I77135,KTXGB1YGLNA5A3MBQWPC,ServiRed,S33T9PXGBMI7GUX051OC,JFD7RA18JJ6YI9L720RE,EURO 6000,Inter-Sistemas,Q7MULZRDPH24847MYMLF,46680,Valencia,VALENCIA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,AJ6YNSIZOH62VR52XFDB,F,NA,0,NA,NA,M,X,0,0,NA,1,50,2016-03-20
4,2016,01,05,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,9GETG4KKGZXQ8PKZSIXT,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,ONF35G4DR4APTKVHMG5A,F,1982-07-20,11100,ESP,ESP,F,C,0,0,1966-03-22,1,20,2016-01-05
5,2016,03,12,8KZOFPY0TDQH5XW7LV8P,SZBCQF5YAO4W2N3ROXI4,EURO 6000,RI7WGGJYVBDG339OG6O3,Q3POXRTGOT0XT7GJTXI8,ServiRed,Inter-Sistemas,SO7T7EIFPQH2FUYZULYC,2005,Albacete,ALBACETE,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2,150,2016-03-12
6,2016,01,05,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,LWM3BYLE6N04FKCD5TYG,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,CrÃ©dito,Reintegros,Debito,OK,8TYVA5CA0VNQ41TFC7QQ,F,1986-05-26,29620,NA,ESP,M,S,0,0,1977-12-04,1,20,2016-01-05
7,2016,01,20,T0HUIQJLCS3MSSJB229H,6OWQM98TUB4TLU818SPA,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,Intra-Sistema,VX95PMMDEA8OYIBHOXCA,18003,Granada,GRANADA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,2XLFJUUN9VKDAVMRBU4Z,F,1994-01-21,29500,ESP,ESP,F,S,0,0,1994-01-21,1,30,2016-01-20
8,2016,01,03,DNYR3O1F7XM5GOOOSLLB,IDFXVIA8S1D7PND1QNPF,ServiRed,S33T9PXGBMI7GUX051OC,JFD7RA18JJ6YI9L720RE,EURO 6000,Inter-Sistemas,Q7MULZRDPH24847MYMLF,31010,Navarra,NAVARRA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,F0NZ5BBOJGSRAVY0LYUC,F,NA,0,NA,NA,M,X,0,0,NA,1,20,2016-01-03
9,2016,02,05,33LCZG4TQMVGQQ2IOQFE,BMENWJCG94XKFNY6CJ55,ServiRed,NKIE64TALWTZ1UHGS17T,X7ZJC37USW0KQYKO68XB,EURO 6000,Inter-Sistemas,BUSM8IDXPSIC74P5S6QU,8930,Barcelona,BARCELONA,ESP,Visa,EstÃ¡ndar,Visa,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,50,2016-02-05
10,2016,01,29,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,5UO3QCLR3WPCSYO3K8JB,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,T30ED28AF5MMY9PJU0JM,F,1977-02-09,29001,ESP,ESP,F,S,0,0,1953-03-28,1,60,2016-01-29


In [48]:
# convert into numeric format
variables <- c('NOPER','IMPOPER')
DT[,(variables):=lapply(.SD, as.numeric),.SDcols=variables]
rm(variables)

,ANO,MES,DIA,OP_ADQUIRENTE,ADQUIERENTE,DES_TIPO_ADQUIRENTE,OP_EMISOR,EMISOR,DES_TIPO_EMISOR,DES_AMBITO,OP_IDENT_TERMINAL,OP_COD_POST_COMERCIO,DES_PROVINCIA,LOCALIDAD,OP_COD_PAIS_COMERCIO,DES_MARCA,DES_GAMA,DES_PRODUCTO,TIPO_TARJETA,DES_CREDEB,DES_CLASE_OPERACION,DES_PAGO,DES_RESULTADO,PER_ID_PERSONA,PER_TIPO_PERS,PER_FECHA_ALTA,OF_COD_POST,PER_COD_PAIS_NAC,OF_COD_PAIS_RES,PER_ID_SEXO,PER_EST_CIVIL,PER_MARCA_EMP,PER_MARCA_FALL,PER_FECHA_NAC,NOPER,IMPOPER,FECHA
1,2016,03,26,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,EUNDITVNRWGASN0VCFMK,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,HWG2BTMLK1WMRVJT3TMU,F,1984-09-26,29002,ESP,ESP,F,C,0,0,1941-03-04,1,30,2016-03-26
2,2016,03,03,BM3MV1QJ1RWI6XB8W36S,Q4SRXYQNPFB8ST2BCSLT,EURO 6000,X4W6L75KAK6TKQFMYXGJ,T08GEJ1FFLLW5WK82Z0M,EURO 6000,On us,HDOHII3H5SMPCZIJI07V,99999,NO EXISTE LA PROVINCIA,NA,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,P,DÃ©bito,Reintegros,Debito,OK,S72O8TEU3YWJFJLXJO7Y,F,1993-12-18,47006,ESP,ESP,F,C,0,0,1945-11-21,1,200,2016-03-03
3,2016,03,20,X4HO0YVLHY4IY6I77135,KTXGB1YGLNA5A3MBQWPC,ServiRed,S33T9PXGBMI7GUX051OC,JFD7RA18JJ6YI9L720RE,EURO 6000,Inter-Sistemas,Q7MULZRDPH24847MYMLF,46680,Valencia,VALENCIA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,AJ6YNSIZOH62VR52XFDB,F,NA,0,NA,NA,M,X,0,0,NA,1,50,2016-03-20
4,2016,01,05,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,9GETG4KKGZXQ8PKZSIXT,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,ONF35G4DR4APTKVHMG5A,F,1982-07-20,11100,ESP,ESP,F,C,0,0,1966-03-22,1,20,2016-01-05
5,2016,03,12,8KZOFPY0TDQH5XW7LV8P,SZBCQF5YAO4W2N3ROXI4,EURO 6000,RI7WGGJYVBDG339OG6O3,Q3POXRTGOT0XT7GJTXI8,ServiRed,Inter-Sistemas,SO7T7EIFPQH2FUYZULYC,2005,Albacete,ALBACETE,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2,150,2016-03-12
6,2016,01,05,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,LWM3BYLE6N04FKCD5TYG,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,CrÃ©dito,Reintegros,Debito,OK,8TYVA5CA0VNQ41TFC7QQ,F,1986-05-26,29620,NA,ESP,M,S,0,0,1977-12-04,1,20,2016-01-05
7,2016,01,20,T0HUIQJLCS3MSSJB229H,6OWQM98TUB4TLU818SPA,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,Intra-Sistema,VX95PMMDEA8OYIBHOXCA,18003,Granada,GRANADA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,2XLFJUUN9VKDAVMRBU4Z,F,1994-01-21,29500,ESP,ESP,F,S,0,0,1994-01-21,1,30,2016-01-20
8,2016,01,03,DNYR3O1F7XM5GOOOSLLB,IDFXVIA8S1D7PND1QNPF,ServiRed,S33T9PXGBMI7GUX051OC,JFD7RA18JJ6YI9L720RE,EURO 6000,Inter-Sistemas,Q7MULZRDPH24847MYMLF,31010,Navarra,NAVARRA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,F0NZ5BBOJGSRAVY0LYUC,F,NA,0,NA,NA,M,X,0,0,NA,1,20,2016-01-03
9,2016,02,05,33LCZG4TQMVGQQ2IOQFE,BMENWJCG94XKFNY6CJ55,ServiRed,NKIE64TALWTZ1UHGS17T,X7ZJC37USW0KQYKO68XB,EURO 6000,Inter-Sistemas,BUSM8IDXPSIC74P5S6QU,8930,Barcelona,BARCELONA,ESP,Visa,EstÃ¡ndar,Visa,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,50,2016-02-05
10,2016,01,29,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,5UO3QCLR3WPCSYO3K8JB,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,T30ED28AF5MMY9PJU0JM,F,1977-02-09,29001,ESP,ESP,F,S,0,0,1953-03-28,1,60,2016-01-29


In [49]:
# Assign NA to DES_PROVINCIA variable with value 'NO EXISTE LA PROVINCIA'
levels(DT$DES_PROVINCIA)[levels(DT$DES_PROVINCIA)=='NO EXISTE LA PROVINCIA'] <- NA

In [50]:
# We reorder de columns to put the date at the begining of each observation
setcolorder(DT, c(ncol(DT), 1:(ncol(DT)-1)))
str(DT)

Classes ‘data.table’ and 'data.frame':	500 obs. of  37 variables:
 $ FECHA               : Date, format: "2016-03-26" "2016-03-03" ...
 $ ANO                 : Factor w/ 1 level "2016": 1 1 1 1 1 1 1 1 1 1 ...
 $ MES                 : Factor w/ 3 levels "01","02","03": 3 3 3 1 3 1 1 1 2 1 ...
 $ DIA                 : Factor w/ 31 levels "01","02","03",..: 26 3 20 5 12 5 20 3 5 29 ...
 $ OP_ADQUIRENTE       : Factor w/ 27 levels "0A86SJKPC5VI5TUKAN01",..: 21 11 25 21 8 21 19 12 2 21 ...
 $ ADQUIERENTE         : chr  "6CVN1DBRKUOUTFT8VBXJ" "Q4SRXYQNPFB8ST2BCSLT" "KTXGB1YGLNA5A3MBQWPC" "6CVN1DBRKUOUTFT8VBXJ" ...
 $ DES_TIPO_ADQUIRENTE : Factor w/ 4 levels "Eufiserv","EURO 6000",..: 2 2 3 2 2 2 2 3 3 2 ...
 $ OP_EMISOR           : chr  "CM8GMN7BQOF9JJ1XXCPE" "X4W6L75KAK6TKQFMYXGJ" "S33T9PXGBMI7GUX051OC" "CM8GMN7BQOF9JJ1XXCPE" ...
 $ EMISOR              : chr  "KSPHEXET1G2LNR4OXAGU" "T08GEJ1FFLLW5WK82Z0M" "JFD7RA18JJ6YI9L720RE" "KSPHEXET1G2LNR4OXAGU" ...
 $ DES_TIPO_EMISOR     : Factor w/ 6

In [51]:
# Transform coded OP_ADQUIRENTE and ADQUIERENTE into a more human friendly string
setkey(DT,OP_ADQUIRENTE)

op_adquiriente <- seq(from = 1000, to=length(unique(DT$OP_ADQUIRENTE))+999, by =1)
DT[,OP_ADQUIRENTE:=factor(OP_ADQUIRENTE,labels=op_adquiriente)]

adquiriente <- paste("Entidad", op_adquiriente)
DT[,ADQUIERENTE:=factor(ADQUIERENTE,labels=adquiriente)]
head(DT, 2)

,FECHA,ANO,MES,DIA,OP_ADQUIRENTE,ADQUIERENTE,DES_TIPO_ADQUIRENTE,OP_EMISOR,EMISOR,DES_TIPO_EMISOR,DES_AMBITO,OP_IDENT_TERMINAL,OP_COD_POST_COMERCIO,DES_PROVINCIA,LOCALIDAD,OP_COD_PAIS_COMERCIO,DES_MARCA,DES_GAMA,DES_PRODUCTO,TIPO_TARJETA,DES_CREDEB,DES_CLASE_OPERACION,DES_PAGO,DES_RESULTADO,PER_ID_PERSONA,PER_TIPO_PERS,PER_FECHA_ALTA,OF_COD_POST,PER_COD_PAIS_NAC,OF_COD_PAIS_RES,PER_ID_SEXO,PER_EST_CIVIL,PER_MARCA_EMP,PER_MARCA_FALL,PER_FECHA_NAC,NOPER,IMPOPER
1,2016-02-10,2016,02,10,1000,C665PU6QMZE5TGSYLCB3,Eufiserv,P67AJ7ISUZ7ZHQV1FMP2,784ENO2000GCDG3KDKKE,EURO 6000,Intercambio Internacional,4EXGNEGXJ7GL9CPJB1GJ,0,NA,NA,PRT,MasterCard,EstÃ¡ndar,MasterCard,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,100
2,2016-02-02,2016,02,02,1000,C665PU6QMZE5TGSYLCB3,Eufiserv,P4T6BK9H823V8CC0XELQ,QF23UI0ICRXGO0ANPKFZ,EURO 6000,Intercambio Internacional,5FA1IKXNH91ACV6IF4KL,0,NA,NA,DEU,Visa,EstÃ¡ndar,Electron/Plus/V Pay,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,20
3,2016-02-05,2016,02,05,1001,BMENWJCG94XKFNY6CJ55,ServiRed,NKIE64TALWTZ1UHGS17T,X7ZJC37USW0KQYKO68XB,EURO 6000,Inter-Sistemas,BUSM8IDXPSIC74P5S6QU,8930,Barcelona,BARCELONA,ESP,Visa,EstÃ¡ndar,Visa,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,50
4,2016-03-14,2016,03,14,1002,87TSBFU6ZVVXZ7Z9XIJC,ServiRed,V84T086QK8PWJ845SZ9P,41AGXLEBTO6WP1PED5JN,Otras Entidades,Otros,W6RCLZ5PXLSKKSRRJR99,41005,Sevilla,SEVILLA,ESP,Otras,EstÃ¡ndar,Otras,NA,CrÃ©dito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2,700
5,2016-01-26,2016,01,26,1002,87TSBFU6ZVVXZ7Z9XIJC,ServiRed,NKIE64TALWTZ1UHGS17T,X7ZJC37USW0KQYKO68XB,EURO 6000,Inter-Sistemas,F00OLL4DHT0QOXX2695U,32005,Ourense,OURENSE,ESP,Visa,EstÃ¡ndar,Visa,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,600
6,2016-01-29,2016,01,29,1002,87TSBFU6ZVVXZ7Z9XIJC,ServiRed,MKV4WRUXCOKUWWTKS3AM,THIW8YV5T3O9OD5HM6Z7,EURO 6000,Inter-Sistemas,DHNJ7Y2X4XEZHTFT7M8T,41900,Sevilla,SEVILLA,ESP,Visa,EstÃ¡ndar,Visa,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,20
7,2016-03-15,2016,03,15,1003,PXBYYHLLV162ZR39MYTX,Sist. 4B,P4T6BK9H823V8CC0XELQ,QF23UI0ICRXGO0ANPKFZ,EURO 6000,Inter-Sistemas,05MOLSRWMYXTDDKZL8VQ,27003,Lugo,LUGO,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,20
8,2016-03-10,2016,03,10,1003,PXBYYHLLV162ZR39MYTX,Sist. 4B,P4T6BK9H823V8CC0XELQ,QF23UI0ICRXGO0ANPKFZ,EURO 6000,Inter-Sistemas,8GCK48A61BH0QLJPI9JS,24400,LeÃ³n,LEÃN,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,40
9,2016-01-23,2016,01,23,1004,34RQ4G37IP4WYO4U3ESA,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,Intra-Sistema,HETMX4MRC8CPZSMBXAAM,14700,CÃ³rdoba,CÃRDOBA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,KAIT47LOVSWQTIQ5JEG4,F,2002-11-12,14700,ESP,ESP,F,C,0,0,1976-07-18,2,180
10,2016-01-27,2016,01,27,1004,34RQ4G37IP4WYO4U3ESA,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,Intra-Sistema,IALLP7BNWRK88ZFURZLE,14880,CÃ³rdoba,CÃRDOBA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,QTZ1AZMEYNJ4KBIFL3NW,F,1999-10-29,14880,ESP,ESP,M,S,0,0,1990-06-18,2,500


,FECHA,ANO,MES,DIA,OP_ADQUIRENTE,ADQUIERENTE,DES_TIPO_ADQUIRENTE,OP_EMISOR,EMISOR,DES_TIPO_EMISOR,DES_AMBITO,OP_IDENT_TERMINAL,OP_COD_POST_COMERCIO,DES_PROVINCIA,LOCALIDAD,OP_COD_PAIS_COMERCIO,DES_MARCA,DES_GAMA,DES_PRODUCTO,TIPO_TARJETA,DES_CREDEB,DES_CLASE_OPERACION,DES_PAGO,DES_RESULTADO,PER_ID_PERSONA,PER_TIPO_PERS,PER_FECHA_ALTA,OF_COD_POST,PER_COD_PAIS_NAC,OF_COD_PAIS_RES,PER_ID_SEXO,PER_EST_CIVIL,PER_MARCA_EMP,PER_MARCA_FALL,PER_FECHA_NAC,NOPER,IMPOPER
1,2016-02-10,2016,02,10,1000,Entidad 1011,Eufiserv,P67AJ7ISUZ7ZHQV1FMP2,784ENO2000GCDG3KDKKE,EURO 6000,Intercambio Internacional,4EXGNEGXJ7GL9CPJB1GJ,0,NA,NA,PRT,MasterCard,EstÃ¡ndar,MasterCard,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,100
2,2016-02-02,2016,02,02,1000,Entidad 1011,Eufiserv,P4T6BK9H823V8CC0XELQ,QF23UI0ICRXGO0ANPKFZ,EURO 6000,Intercambio Internacional,5FA1IKXNH91ACV6IF4KL,0,NA,NA,DEU,Visa,EstÃ¡ndar,Electron/Plus/V Pay,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,20
3,2016-02-05,2016,02,05,1001,Entidad 1010,ServiRed,NKIE64TALWTZ1UHGS17T,X7ZJC37USW0KQYKO68XB,EURO 6000,Inter-Sistemas,BUSM8IDXPSIC74P5S6QU,8930,Barcelona,BARCELONA,ESP,Visa,EstÃ¡ndar,Visa,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,50
4,2016-03-14,2016,03,14,1002,Entidad 1008,ServiRed,V84T086QK8PWJ845SZ9P,41AGXLEBTO6WP1PED5JN,Otras Entidades,Otros,W6RCLZ5PXLSKKSRRJR99,41005,Sevilla,SEVILLA,ESP,Otras,EstÃ¡ndar,Otras,NA,CrÃ©dito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2,700
5,2016-01-26,2016,01,26,1002,Entidad 1008,ServiRed,NKIE64TALWTZ1UHGS17T,X7ZJC37USW0KQYKO68XB,EURO 6000,Inter-Sistemas,F00OLL4DHT0QOXX2695U,32005,Ourense,OURENSE,ESP,Visa,EstÃ¡ndar,Visa,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,600
6,2016-01-29,2016,01,29,1002,Entidad 1008,ServiRed,MKV4WRUXCOKUWWTKS3AM,THIW8YV5T3O9OD5HM6Z7,EURO 6000,Inter-Sistemas,DHNJ7Y2X4XEZHTFT7M8T,41900,Sevilla,SEVILLA,ESP,Visa,EstÃ¡ndar,Visa,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,20
7,2016-03-15,2016,03,15,1003,Entidad 1021,Sist. 4B,P4T6BK9H823V8CC0XELQ,QF23UI0ICRXGO0ANPKFZ,EURO 6000,Inter-Sistemas,05MOLSRWMYXTDDKZL8VQ,27003,Lugo,LUGO,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,20
8,2016-03-10,2016,03,10,1003,Entidad 1021,Sist. 4B,P4T6BK9H823V8CC0XELQ,QF23UI0ICRXGO0ANPKFZ,EURO 6000,Inter-Sistemas,8GCK48A61BH0QLJPI9JS,24400,LeÃ³n,LEÃN,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,40
9,2016-01-23,2016,01,23,1004,Entidad 1005,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,Intra-Sistema,HETMX4MRC8CPZSMBXAAM,14700,CÃ³rdoba,CÃRDOBA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,KAIT47LOVSWQTIQ5JEG4,F,2002-11-12,14700,ESP,ESP,F,C,0,0,1976-07-18,2,180
10,2016-01-27,2016,01,27,1004,Entidad 1005,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,Intra-Sistema,IALLP7BNWRK88ZFURZLE,14880,CÃ³rdoba,CÃRDOBA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,NA,OK,QTZ1AZMEYNJ4KBIFL3NW,F,1999-10-29,14880,ESP,ESP,M,S,0,0,1990-06-18,2,500


,FECHA,ANO,MES,DIA,OP_ADQUIRENTE,ADQUIERENTE,DES_TIPO_ADQUIRENTE,OP_EMISOR,EMISOR,DES_TIPO_EMISOR,DES_AMBITO,OP_IDENT_TERMINAL,OP_COD_POST_COMERCIO,DES_PROVINCIA,LOCALIDAD,OP_COD_PAIS_COMERCIO,DES_MARCA,DES_GAMA,DES_PRODUCTO,TIPO_TARJETA,DES_CREDEB,DES_CLASE_OPERACION,DES_PAGO,DES_RESULTADO,PER_ID_PERSONA,PER_TIPO_PERS,PER_FECHA_ALTA,OF_COD_POST,PER_COD_PAIS_NAC,OF_COD_PAIS_RES,PER_ID_SEXO,PER_EST_CIVIL,PER_MARCA_EMP,PER_MARCA_FALL,PER_FECHA_NAC,NOPER,IMPOPER
1,2016-02-10,2016,02,10,1000,Entidad 1011,Eufiserv,P67AJ7ISUZ7ZHQV1FMP2,784ENO2000GCDG3KDKKE,EURO 6000,Intercambio Internacional,4EXGNEGXJ7GL9CPJB1GJ,0,NA,NA,PRT,MasterCard,EstÃ¡ndar,MasterCard,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,100
2,2016-02-02,2016,02,02,1000,Entidad 1011,Eufiserv,P4T6BK9H823V8CC0XELQ,QF23UI0ICRXGO0ANPKFZ,EURO 6000,Intercambio Internacional,5FA1IKXNH91ACV6IF4KL,0,NA,NA,DEU,Visa,EstÃ¡ndar,Electron/Plus/V Pay,NA,DÃ©bito,Reintegros,NA,OK,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,20


In [52]:
# Filter out all the operations where PER_ID_PERSONA is NA and DES_RESULTADO is not OK
DT <- DT[!is.na(PER_ID_PERSONA) & DES_RESULTADO == "OK"]

In [53]:
# Save the data table into a serialized object for later usage. This will speed up later porcesses of merging the resulting data set with the cluster variable with the original one.
saveRDS(DT, file = "./data/DT.rds", compress = T)

## Feature creation
New variables are derived from the previous ones. These variables are the final ones after an iterative process of pruning:
* High correlated variables
* Zero variance predictors or with linear dependencies: that is, variables with few unique values relative to other values within feature, or near zero variance or high frequency of most common value or mode value.

In [54]:
setkeyv(DT,c("PER_ID_PERSONA","MES"))
DT <- DT[, list(F1=median(na.omit(IMPOPER[which(DES_AMBITO == "On us")])),
                F2=median(na.omit(IMPOPER[which(DES_AMBITO == "Inter-Sistemas")])),
                F3=median(na.omit(IMPOPER[which(DES_AMBITO == "Intra-Sistema")]), na.rm = T),
                F4=length(unique(.N[which(DES_AMBITO == "On us")])),
                F5=length(unique(.N[which(DES_AMBITO == "Inter-Sistemas")])),
                F6=length(unique(.N[which(DES_AMBITO == "Intra-Sistema")]))               
),
by=.(PER_ID_PERSONA)]

In [55]:
# Remove any NA observation. Altough none of the previous derived features have NA values. Just for be sure.
DT[is.na(DT)] <- 0

In [56]:
# The data set obtained frim previous operations
head(DT, 2)

,PER_ID_PERSONA,F1,F2,F3,F4,F5,F6
1,010BPUWEHI43K9AB179N,400,0,0,1,0,0
2,0GQ3NQMI1N24JE0499AY,10,0,0,1,0,0


## Data normalization


In [60]:
# data.table data frames doesn't suppor assigning row names. This is a cool and interesting feature in R data frames. We can assign each row an unique labes with the PER_ID_PERSONA and join later the assigned cluster to each PER_ID_PERSONA
# Convert the data.table to a data.frame
DT <- as.data.frame(DT)
# Set each row names as the PER_ID_PERSONA 
row.names(DT) <- DT$PER_ID_PERSONA
# Delete de PER_ID_PERSONA variable, no needed for the clustering algorithm
DT$PER_ID_PERSONA <- NULL

In [61]:
# Data normalization
DT <- scale(DT)
head(DT, 2)

F1,F2,F3,F4,F5,F6
2.1703465,-0.1020868,-0.2631964,0.5349538,-0.2944623,-0.4218309
-0.5424624,-0.1020868,-0.2631964,0.5349538,-0.2944623,-0.4218309


## Export

In [62]:
# write de data.frame as a CSV file ready to be uploaded clustered
write.table(DT[complete.cases(DT),], "./data/sample_scaled.csv", row.names = T, col.names = FALSE, sep=",")